In [49]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

In [50]:
df = pd.read_csv('downtown_restaurants_yelp (3).csv')

In [51]:
df.head()

,web-scraper-order,web-scraper-start-url,restaurant_name_yelp,restaurant_name_yelp-href,restaurant_name_text_yelp,comments_yelp,reviews_yelp,pages,pages-href,pages2,pages2-href,Dollar,Type
0,1633289047-11083,https://www.yelp.com/search?cflt=restaurants&f...,Corner,https://www.yelp.com/biz/corner-austin-2,Corner,NaN,NaN,NaN,NaN,2.0,https://www.yelp.com/biz/corner-austin-2#,$$,NaN
1,1633288363-7285,https://www.yelp.com/search?cflt=restaurants&f...,Bangers Sausage House & Beer Garden,https://www.yelp.com/biz/bangers-sausage-house...,Bangers Sausage House & Beer Garden,NaN,NaN,9.0,https://www.yelp.com/search?cflt=restaurants&f...,2.0,https://www.yelp.com/biz/bangers-sausage-house...,$$,Breakfast & Brunch
2,1633288736-9353,https://www.yelp.com/search?cflt=restaurants&f...,Rosewood,https://www.yelp.com/biz/rosewood-austin-3,Rosewood,I wish I could give more than 5 stars! Came he...,NaN,5.0,https://www.yelp.com/search?cflt=restaurants&f...,NaN,NaN,$$$,NaN
3,1633288464-7843,https://www.yelp.com/search?cflt=restaurants&f...,South Congress Cafe,https://www.yelp.com/biz/south-congress-cafe-a...,South Congress Cafe,We saw a ton of recommendations for this place...,NaN,8.0,https://www.yelp.com/search?cflt=restaurants&f...,NaN,NaN,$$,NaN
4,1633288793-9671,https://www.yelp.com/search?cflt=restaurants&f...,Eddie V’s Prime Seafood,https://www.yelp.com/biz/eddie-vs-prime-seafoo...,Eddie V's Prime Seafood,NaN,NaN,4.0,https://www.yelp.com/search?cflt=restaurants&f...,NaN,NaN,$$$,Seafood


# Data Cleaning  

In [52]:
df = df.loc[:,['restaurant_name_yelp', 'comments_yelp']]
print(df.shape)
df = df.dropna()
df = df.drop_duplicates()
print(df.shape)
df.head()

(4125, 2)
(951, 2)


,restaurant_name_yelp,comments_yelp
2,Rosewood,I wish I could give more than 5 stars! Came he...
3,South Congress Cafe,We saw a ton of recommendations for this place...
5,BAMBU,I use to come here all the time like 2 years a...
7,Arepa Dealers,Tasted this magical goodness once again at the...
9,REVUE,Pizza took SO long to cook and once we finally...


# Tokenization 

In [53]:
import nltk
com_df = df.copy()
## This tokenizer will remove the punctuation marks from the data automatically
tokenizer = nltk.RegexpTokenizer("\w+\'*\w+")

com_df['tokens'] = com_df['comments_yelp'].map(lambda x: tokenizer.tokenize(str(x)))
com_df['tokens'] = com_df['tokens'].map(lambda x: [a.lower() for a in x])
com_df['len_token'] = com_df['tokens'].map(lambda x: len(x))
com_df.head()

,restaurant_name_yelp,comments_yelp,tokens,len_token
2,Rosewood,I wish I could give more than 5 stars! Came he...,"[wish, could, give, more, than, stars, came, h...",117
3,South Congress Cafe,We saw a ton of recommendations for this place...,"[we, saw, ton, of, recommendations, for, this,...",185
5,BAMBU,I use to come here all the time like 2 years a...,"[use, to, come, here, all, the, time, like, ye...",165
7,Arepa Dealers,Tasted this magical goodness once again at the...,"[tasted, this, magical, goodness, once, again,...",92
9,REVUE,Pizza took SO long to cook and once we finally...,"[pizza, took, so, long, to, cook, and, once, w...",37


In [54]:
from collections import Counter

In [55]:
ls = list([])
for x in com_df['tokens']:
    ls = ls+x
print(len(ls))

116174


In [56]:
unq_tokens = list(set(ls))
freq_list = [ls.count(x) for x in unq_tokens]

freq_df = pd.DataFrame({
    'token': unq_tokens,
    'frequency': freq_list
})
freq_df.head()

,token,frequency
0,me,313
1,kit,3
2,curved,1
3,rippable,1
4,noted,4


In [57]:
from nltk.corpus import stopwords
nltk.download('stopwords')
freq_df_stop_words_removed = freq_df.loc[~freq_df['token'].isin(stopwords.words('english')),:]
freq_df_stop_words_removed = freq_df_stop_words_removed.loc[:,['token','frequency']]
freq_df_stop_words_removed.head(5)

[nltk_data] Downloading package stopwords to /Users/Kaushik
[nltk_data]     Kumaran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,token,frequency
1,kit,3
2,curved,1
3,rippable,1
4,noted,4
5,compote,1


In [58]:
freq_df_stop_words_removed.sort_values(by = 'frequency', ascending = False).head(30)

,token,frequency
5299,food,756
2051,good,659
401,great,554
6950,place,431
6113,service,377
3614,really,376
4233,like,357
3806,one,346
6894,also,346
4741,would,335


# List of Attributes that we consider as important 

service

drinks

experience 

In [59]:
import spacy 
nlp = spacy.load("en_core_web_md")
import warnings; warnings.simplefilter('ignore')

In [60]:
def func_1(inp_ls):
    ls = [x for x in inp_ls if x not in stopwords.words('english') ]
    b = ' '.join(ls)
    return b

com_df['comments_removed_stop_words'] = com_df['tokens'].apply(func_1)
com_df.head()

,restaurant_name_yelp,comments_yelp,tokens,len_token,comments_removed_stop_words
2,Rosewood,I wish I could give more than 5 stars! Came he...,"[wish, could, give, more, than, stars, came, h...",117,wish could give stars came birthday fantastic ...
3,South Congress Cafe,We saw a ton of recommendations for this place...,"[we, saw, ton, of, recommendations, for, this,...",185,saw ton recommendations place online looking b...
5,BAMBU,I use to come here all the time like 2 years a...,"[use, to, come, here, all, the, time, like, ye...",165,use come time like years ago came recently cal...
7,Arepa Dealers,Tasted this magical goodness once again at the...,"[tasted, this, magical, goodness, once, again,...",92,tasted magical goodness new location must say ...
9,REVUE,Pizza took SO long to cook and once we finally...,"[pizza, took, so, long, to, cook, and, once, w...",37,pizza took long cook finally got cold really d...


In [61]:
user_comment = "spicy chicken"
doc2 = nlp(user_comment)

In [62]:
def find_similarity(review):
    review_nlp = nlp(review)
    sim_score = review_nlp.similarity(doc2)
    return sim_score
com_df['sim_score'] = com_df['comments_removed_stop_words'].apply(find_similarity)
com_df.head()

,restaurant_name_yelp,comments_yelp,tokens,len_token,comments_removed_stop_words,sim_score
2,Rosewood,I wish I could give more than 5 stars! Came he...,"[wish, could, give, more, than, stars, came, h...",117,wish could give stars came birthday fantastic ...,0.631149
3,South Congress Cafe,We saw a ton of recommendations for this place...,"[we, saw, ton, of, recommendations, for, this,...",185,saw ton recommendations place online looking b...,0.500828
5,BAMBU,I use to come here all the time like 2 years a...,"[use, to, come, here, all, the, time, like, ye...",165,use come time like years ago came recently cal...,0.460294
7,Arepa Dealers,Tasted this magical goodness once again at the...,"[tasted, this, magical, goodness, once, again,...",92,tasted magical goodness new location must say ...,0.571030
9,REVUE,Pizza took SO long to cook and once we finally...,"[pizza, took, so, long, to, cook, and, once, w...",37,pizza took long cook finally got cold really d...,0.512641


In [63]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [ ]:
def model_bert_similarity(review):    
    e1 = model.encode(review)
    e2 = model.encode(user_comment)
    return util.cos_sim(e1,e2).numpy()[0][0]


com_df['similarity_BERT'] = com_df['comments_removed_stop_words'].apply(model_bert_similarity)
com_df.head()

In [ ]:
com_df.sort_values(by = ['similarity_BERT'], ascending = False).iloc[2]['comments_yelp']

# Sentiment Analysis 

In [ ]:
## Keeping top 50 comments based on the similarity score
num_comments = 100
similarity_column = 'sim_score'
Top = com_df.sort_values(by = [similarity_column], ascending = False).head(num_comments)
Top.head()
#Top.shape

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def get_sentiment(review, **kwargs):
    sentiment_score = analyser.polarity_scores(review)
    positive_meter = round((sentiment_score['pos']),2)
    negative_meter = round((sentiment_score['neg']), 2)
    return positive_meter if kwargs['k'] == 'positive' else negative_meter

In [ ]:
Top['positive'] = Top['comments_yelp'].apply(get_sentiment, k='positive')
Top['negative'] = Top['comments_yelp'].apply(get_sentiment, k='negative')

Top['pos_scaled']  = Top['positive']/max(Top['positive'])
Top[similarity_column]  = Top[similarity_column]/max(Top[similarity_column])


In [ ]:
Top['combination'] = (Top[similarity_column]*1.5 * Top['pos_scaled'])/(2.5)
Top.sort_values(by = ['combination'], ascending= False).iloc[5]['comments_yelp']

In [ ]:
Top.sort_values(by = ['combination'], ascending = False).iloc[8]['comments_yelp']

In [ ]:
Top.sort_values(by = ['combination'], ascending = False)

# Topic Modelling

In [ ]:
my_cv = CountVectorizer(tokenizer=clean_tokenize_lemmatize_tweet)
my_lda = LatentDirichletAllocation(n_components=5)
 
my_dtm = cv.fit_transform(sample_corpus)
my_lda.fit(my_dtm) # may take a while with a large corpus ;)